In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torchvision import datasets, transforms

In [0]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.fc = nn.Linear(64 * 12 ** 2, 10)
        self.layers = torch.nn.Sequential(
            nn.Conv2d(1, 32, 3, 1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, 1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )

    def forward(self, x):
        """
        :param x: torch.Size([bs, 1, 28, 28])
        """
        x = self.layers(x)
        x = self.fc(x.view(len(x), -1))
        return F.log_softmax(x, dim=1)

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 256
lr = 0.0001
epochs = 10
torch.cuda.manual_seed(1234)

In [0]:
transform = transforms.Compose([
     transforms.ToTensor(),
     transforms.Normalize((0.1307,), (0.3081,))
])

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('.', train=True, download=True, transform=transform),
    batch_size=batch_size, shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('.', train=False, transform=transform),
    batch_size=batch_size, shuffle=True
)

In [0]:
model = NeuralNet().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

In [0]:
def train(model, device, dataloader, optimizer, epoch):
    model.train()
    loss_val = 0
    for batch in train_loader:
        input, target = [e.to(device) for e in batch]
        output = model(input)
        loss = F.nll_loss(output, target)
        loss_val += loss.item() * len(input)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    print(f"Train {epoch} epoch |"
          f"loss: {loss_val / len(dataloader.dataset):.3f}")

In [0]:
def valid(model, device, dataloader, epoch):
    model.eval()
    loss_val, accuracy = 0, 0
    with torch.no_grad():
        for batch in dataloader:
            input, target = [e.to(device) for e in batch]
            output = model(input)
            loss_val += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            accuracy += pred.eq(target.view_as(pred)).sum().item()

    print(f"Test {epoch} epoch |"
          f"loss: {loss_val / len(dataloader.dataset):.3f} |"
          f"accuracy: {accuracy / len(dataloader.dataset):.3f}\n")

In [144]:
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    valid(model, device, test_loader, epoch)

Train 1 epoch |loss: 0.548
Test 1 epoch |loss: 0.235 |accuracy: 0.935

Train 2 epoch |loss: 0.198
Test 2 epoch |loss: 0.141 |accuracy: 0.959

Train 3 epoch |loss: 0.127
Test 3 epoch |loss: 0.094 |accuracy: 0.974

Train 4 epoch |loss: 0.093
Test 4 epoch |loss: 0.075 |accuracy: 0.979

Train 5 epoch |loss: 0.075
Test 5 epoch |loss: 0.061 |accuracy: 0.982

Train 6 epoch |loss: 0.063
Test 6 epoch |loss: 0.054 |accuracy: 0.983

Train 7 epoch |loss: 0.055
Test 7 epoch |loss: 0.050 |accuracy: 0.984

Train 8 epoch |loss: 0.050
Test 8 epoch |loss: 0.049 |accuracy: 0.984

Train 9 epoch |loss: 0.046
Test 9 epoch |loss: 0.045 |accuracy: 0.985

Train 10 epoch |loss: 0.043
Test 10 epoch |loss: 0.044 |accuracy: 0.985



In [0]:
def attack(image, epsilon, grad):
    attacked_image = image + epsilon * grad.sign()
    return attacked_image

In [0]:
def adversarial_test(model, device, dataloader, epsilon):
    accuracy = 0
    for batch in test_loader:
        input, target = [e.to(device) for e in batch]
        input.requires_grad = True
        output = model(input)
        loss = F.nll_loss(output, target)
        loss.backward()
        grad = input.grad.data
        attacked_data = attack(input, epsilon, input.grad.data)
        model.zero_grad()
        # re-classify
        output = model(attacked_data)
        pred = output.argmax(dim=1, keepdim=True)
        accuracy += pred.eq(target.view_as(pred)).sum().item()

    print(f"Epsilon: {epsilon:.2f} |"
          f"Accuracy: {accuracy / len(dataloader.dataset):.3f}")

## Atack

In [165]:
for epsilon in np.linspace(0., 0.5, 11):
    adversarial_test(model, device, test_loader, epsilon)

Epsilon: 0.00 |Accuracy: 0.985
Epsilon: 0.05 |Accuracy: 0.971
Epsilon: 0.10 |Accuracy: 0.950
Epsilon: 0.15 |Accuracy: 0.910
Epsilon: 0.20 |Accuracy: 0.844
Epsilon: 0.25 |Accuracy: 0.747
Epsilon: 0.30 |Accuracy: 0.607
Epsilon: 0.35 |Accuracy: 0.434
Epsilon: 0.40 |Accuracy: 0.275
Epsilon: 0.45 |Accuracy: 0.145
Epsilon: 0.50 |Accuracy: 0.071
